<a href="https://colab.research.google.com/github/johanjun/fc_mobis_4th/blob/main/NCAP_Crawler_ColabVer(%ED%95%9C%EC%A4%80)_fromscratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import zipfile
import time
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd ## TODO: Look into using pandas for JSON parsing
import tkinter as tk
from tkinter.filedialog import askdirectory

import random
import time # Sleep을 위해...


# from selenium import webdriver
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions

In [42]:
cd /content/drive/MyDrive

/content/drive/MyDrive


In [3]:
# 기초함수 (TEST No, Vehicle ID 등 뽑아내는 함수들)

def get_vehicle_info(api_endpoint):
    get_vehicle = requests.get(vehicle_api + api_endpoint)
    vehicle_dict = json.loads(get_vehicle.text)
    vehicle_result = vehicle_dict["Results"]

    return vehicle_result


In [4]:
def get_test_id(test_info):

    test_mode = ["FrontCrashPicture", "SideCrashPicture", "SidePolePicture"]
    test_id = []
    having_mode = []

    for i in test_mode:
      try :
        test_photo = test_info[i]
        test_id.append(test_photo[test_photo.rindex("/")+2:test_photo.rindex("P",0,test_photo.rindex("."))])
        # jpg확장자가 대문자(JPG)로 되어있을 경우 테스트 id가 이상하게됨.
        having_mode.append(i[0:-7])
      except :
        print(i[0:-7], "빈 모드입니다.")
        
    print("성공, test_id : ", test_id, "having mode :", having_mode)
         
    return test_id, having_mode

In [65]:
# Download test data
def get_data(temp_test_id, temp_test_path):
    data_url = f"https://www-nrd.nhtsa.dot.gov/database/VSR/Download.aspx?tstno={temp_test_id}&curno=&database=v&name=v{temp_test_id}&format="
    crash_data_formats = ["json"]
    data_path = f"{temp_test_path}/data"

    for i in range(len(crash_data_formats)):
        url = data_url + crash_data_formats[i]
        r = requests.get(url)
        print(f"{temp_test_id} {i} file URL {url}")
        os.mkdir(data_path)
       
        with open(f"{data_path}/v{temp_test_id}{crash_data_formats[i]}.zip", "wb") as f:
            f.write(r.content)
            
    unzip_data(temp_test_id, data_path)

#Unzip            
def unzip_data(temp_test_id, temp_data_path):
    print('Unzip 시도')
    for i in range(len(temp_test_id)):    
        zip_list = os.listdir(temp_data_path)
        for i in range(len(zip_list)):
            zip_path = temp_data_path + '/' + zip_list[i]
            try :
                with zipfile.ZipFile(zip_path, 'r') as unzip:
                    unzip.extractall(temp_data_path)
                print("Unzip 성공")
            except :
                print("Unzip 실패 : ", zip_path)
                pass

In [63]:
#폴더 만드는 함수

def folder_make(save_path, selected_car, test_id, having_mode ):

  selected_year = selected_car[0]
  selected_make = selected_car[1]
  selected_model = selected_car[2]

  # Top level folder: [Model Year] [Make] [Model]
  lev1_folder = f"{save_path}/{selected_year.upper()} {selected_make.upper()} {selected_model.upper()}"

  try:
    print("lev1시도 : ", lev1_folder)
    os.mkdir(lev1_folder)
    time.sleep(3)
    print("lev1성공, 저장된 폴더 경로 : ", lev1_folder)
  except:
    print("lev1실패")
    pass

  # Second Level folder: [Test ID] - [Model Year] [Make] [Model] - [Test Mode]
  lev2_folder = []
  for i in range(len(having_mode)):
    lev2_folder.append(f"{lev1_folder}/{test_id[i]} - {having_mode[i]} - {selected_year.upper()} {selected_make.upper()} {selected_model.upper()}")

  for x in lev2_folder:
    try:
      print("lev2시도 : ", x)
      os.mkdir(x)
      time.sleep(3)
      print("lev2성공, 저장된 폴더 경로 : ", x)
    except:
      print("lev2실패")
      pass

  return lev2_folder
  # front_folder = lev2_folder[0]
  # side_mdb_folder = lev2_folder[1]
  # side_pole_folder = lev2_folder[2]

In [9]:
vehicle_api = "https://vpic.nhtsa.dot.gov/api"
safety_ratings_api = "https://webapi.nhtsa.gov/api/SafetyRatings"
crash_database = "https://www-nrd.nhtsa.dot.gov/database/VSR/veh/TestDetail.aspx?"
vehicle_makes_api = "/vehicles/GetMakesForVehicleType/car?format=json"

In [15]:
# 1. 입력받은 모델 연도로 부터 메이커 추출

df = pd.DataFrame()

for selected_year in [2017]:
    
  vehicle_id_api = f"/modelyear/{selected_year}?format=json"
  get_vehicle_id = requests.get(safety_ratings_api + vehicle_id_api)
  vehicle_id_dict = json.loads(get_vehicle_id.text)
  vehicle_id_result = vehicle_id_dict["Results"]

  print('\n vehicle_id_api', vehicle_id_api)
  print('\n get_vehicle_id', get_vehicle_id)
  print('\n vehicle_id_dict', vehicle_id_dict)
  print('\n vehicle_id_result', vehicle_id_result)

  print(f"모델연도 {selected_year}에 시험한 자동차 메이커의 수 : {len(vehicle_id_result)}")


  df_make = pd.DataFrame()
  for i in range(len(vehicle_id_result)):
    df_make=df_make.append(vehicle_id_result[i],ignore_index=True)
      
  for selected_make in df_make['Make']:
    vehicle_id_api = f"/modelyear/{selected_year}/make/{selected_make}?format=json"
    get_vehicle_id = requests.get(safety_ratings_api + vehicle_id_api)
    vehicle_id_dict = json.loads(get_vehicle_id.text)
    vehicle_id_result = vehicle_id_dict["Results"]

      
    for i in range(len(vehicle_id_result)):
        df=df.append(vehicle_id_result[i],ignore_index=True)
      
  display(df)


 vehicle_id_api /modelyear/2017?format=json

 get_vehicle_id <Response [200]>

 vehicle_id_dict {'Count': 39, 'Message': 'Results returned successfully', 'Results': [{'ModelYear': 2017, 'Make': 'ACURA', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'ALFA ROMEO', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'AUDI', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'BENTLEY', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'BMW', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'BUICK', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'CADILLAC', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'CHEVROLET', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'CHRYSLER', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'DODGE', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'FIAT', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'FORD', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'GENESIS', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'GMC', 'VehicleId': 0}, {'ModelYear': 2017, 'Make': 'HONDA', 'VehicleId': 0}, {'ModelYear

,Make,Model,ModelYear,VehicleId
0,ACURA,ILX,2017.0,0.0
1,ACURA,MDX,2017.0,0.0
2,ACURA,MDX SPORT HYBRID,2017.0,0.0
3,ACURA,NSX,2017.0,0.0
4,ACURA,RDX,2017.0,0.0
...,...,...,...,...
487,VOLVO,V90 CC,2017.0,0.0
488,VOLVO,XC60,2017.0,0.0
489,VOLVO,XC90 T5,2017.0,0.0
490,VOLVO,XC90 T6,2017.0,0.0


In [66]:
for i in range(len(df[df['ModelYear']==2017][:5])):
  selected_car = [str(int(df.iloc[i]['ModelYear'])),
                  df.iloc[i]['Make'],
                  df.iloc[i]['Model']
                ]
  
  selected_year = selected_car[0]
  selected_make = selected_car[1]
  selected_model = selected_car[2]
  save_path = '/content/drive/MyDrive/Crash/data'
  save_path = os.path.join(save_path, selected_year)

  print("***"*20, f"{len(df.index)} 중 {i+1} 번째 ", "***"*20)
  print(f"selected_year : {selected_year} / selected_make : {selected_make} / selected_model : {selected_model} / save_path : {save_path}")

  vehicle_models_api = f"/vehicles/GetModelsForMakeYear/make/{selected_make}/modelyear/{selected_year}?format=json"
  models_result = get_vehicle_info(vehicle_models_api)

  vehicle_id_api = f"/modelyear/{selected_year}/make/{selected_make}/model/{selected_model}?format=json"
  print("vehicle_id_api : ", vehicle_id_api)
  
  try :

    get_vehicle_id = requests.get(safety_ratings_api + vehicle_id_api)
    vehicle_id_dict = json.loads(get_vehicle_id.text)
    vehicle_id_result = vehicle_id_dict["Results"]

    vehicle_id_set = set()

    for x in vehicle_id_result:
        vehicle_id_set.add(x["VehicleId"])
        print("\n VehicleId : ", x['VehicleId'])

    vehicle_id = vehicle_id_set.pop()
    print("\n vehicle_id : ", vehicle_id)

    test_info_api = f"/VehicleId/{vehicle_id}?format=json"
    
    df.loc[i, 'VehicleId'] = vehicle_id 

    get_test_info = requests.get(safety_ratings_api + test_info_api)
    test_info_dict = json.loads(get_test_info.text)
    test_info_result = test_info_dict["Results"][0]

    test_id, having_mode = get_test_id(test_info_result)

    #함수 호출 부
    lev2_folder = folder_make(save_path, selected_car, test_id, having_mode) #폴더를 만들고 lev2 폴더 경로를 반환함
    print("lev2_folder : ", lev2_folder)
    for i in range(len(test_id)):
      print("get_data : ", test_id[i], lev2_folder[i])
      get_data(test_id[i], lev2_folder[i])
      time.sleep(random.uniform(1,5))     
          
  except : 
    print("해당차종은 오류가 발생하였습니다.")
    pass

************************************************************ 492 중 1 번째  ************************************************************
selected_year : 2017 / selected_make : ACURA / selected_model : ILX / save_path : /content/drive/MyDrive/Crash/data/2017
vehicle_id_api :  /modelyear/2017/make/ACURA/model/ILX?format=json

 VehicleId :  11015

 vehicle_id :  11015
성공, test_id :  ['09252', '09254', '09253'] having mode : ['FrontCrash', 'SideCrash', 'SidePole']
lev1시도 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX
lev1성공, 저장된 폴더 경로 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX
lev2시도 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX/09252 - FrontCrash - 2017 ACURA ILX
lev2성공, 저장된 폴더 경로 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX/09252 - FrontCrash - 2017 ACURA ILX
lev2시도 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX/09254 - SideCrash - 2017 ACURA ILX
lev2성공, 저장된 폴더 경로 :  /content/drive/MyDrive/Crash/data/2017/2017 ACURA ILX/09254 - SideCr